## Dependencies

In [229]:
# Confirmed needed dependencies
import pandas as pd
import sys

# Dependencies for mapping
import gmplot

# Dependency for Heat Mapper
import gmaps

from config import google_API_Key

from ipywidgets.embed import embed_minimal_html

!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


## File inputs/outputs

In [230]:
randLatLon_csv = "./Data/Archived/randomLatLon.csv" 
addressList_csv = "./Data/Archived/addressList.csv"
masterDataCLEAN_csv = "./Data/masterDataCLEAN.csv"
masterDataCLEAN2_csv = "./Data/masterDataCLEAN2.csv" # with crime statistics

## Helpful Functions

In [231]:
#############################
##### VALERIE'S BLOCKS #####
###########################

# Funtion for reading CSV in as DataFrame
def csvDF(oldCSVfilepath):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    return DF

# Function for converting DataFrame to CSV
def DFcsv(dataframe, newCSVfilepath):
    dataframe.to_csv(newCSVfilepath, index=False, header=True)
    print(f"Successfully written to '{newCSVfilepath}'")
    
# Function for reading in csv, checking for headers, and appending if appropriate
def csvDFappend(oldCSVfilepath, newColumn):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    # Checking to ensure new header name does not match any current headers
    colNames = DF.columns
    for value in colNames:
        if value == newColumn:
            print("Cannot append column that matches an existing column name")
            return DF
    # Check to ensure length of newColumn matches length of current dataframe columns
    if len(newColumn) != len(DF):
        print("Cannot append column that is not the same length as existing dataframe")
        return DF
    # Append newColumn to Dataframe
    DF[newColumn] = newColumn
    return DF

## Heat Mapper

In [5]:
# Troy's section


gmaps.configure(api_key=google_API_Key)

In [157]:
# This cell uses gmaps library to create a google heat map from the data in a master data file.
# The masterData csv file is taken as input
# The lat and lon columns are taken as the coordinates for hte heatmap 
# The user specified column is taken as the weighting valies fo each coordinate point

df = masterData_df
columnToMap = 'valueToMap'
max_intensity = df[columnToMap].max()

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
fig.add_layer(heatmap_layer)
fig

NameError: name 'masterData_df' is not defined

In [ ]:
#####This is currently not working as a fucntion. for expediency I suggest we past teh heatmapper code into a cell as needed

# # this is a function version of the cell above
# # the function takes columnToMap as the weights for the points defined by 'lat' and 'lon' columns in the dataframe
# # the dataframe can be included as a parameter, if it is not included masterData_df is assumed

# def heatMapper(columnToMap, df = masterData_df):
    
#     max_intensity = df[columnToMap].max()
    
#     fig = gmaps.figure()
#     heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
#     fig.add_layer(heatmap_layer)

#     return;

In [ ]:
heatMapper(columnToMap = 'valueToMap')
fig

## Commute Time Analysis

In [245]:
# Troy's section
# This section will prepare the data and display on a heatmap

# read in masterdata file into dataframe

data_df = pd.read_csv(masterDataCLEAN2_csv)

# converting commute time to a score value with the longest commute in the data set being scored 0 and the shortest commute being scored 100
min = data_df.commuteTime.min()
max = data_df.commuteTime.max()

data_df["commuteScore"] =100-((data_df.commuteTime-min)*(100/(max-min)))

data_df.head()


,Unnamed: 0,Zillow ID,address,zipcode,alat,alon,valuation,sqft,value sqft,neighborhood,tractCode,countyFips,stateFips,commuteTime,Region,Severity,commuteScore
0,0,29498621,8113 West Gate Boulevard,78745,30.197589,-97.828106,265572,940,282.52,Cherry Creek,1728,453,48,27.7,H7,476,46.048110
1,1,29440641,2109 Red Stone Lane,78727,30.418454,-97.696190,249696,1359,183.74,Tomanet Estates,1848,453,48,21.4,O12,970,67.697595
2,2,29345444,9 Glen Rock Drive,78738,30.341187,-97.984049,472040,2896,163.00,NaN,1773,453,48,30.8,M2,0,35.395189
3,3,29613796,8333 Alvin High Lane,78729,30.460441,-97.761118,307384,1983,155.01,Hunter's Chase,20411,491,48,27.0,Q10,62,48.453608
4,4,29358809,5300 Valburn Circle,78731,30.374096,-97.775259,941669,4376,215.19,North Cat Mountain,1705,453,48,20.3,N9,300,71.477663


In [294]:
# Creating a heat map for the Commute scores

df = data_df
columnToMap = 'commuteScore'
point_radius = 10

figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}


fig = gmaps.figure(layout=figure_layout)
commute_heatmap_layer = gmaps.heatmap_layer(df[['alat', 'alon']], weights=df[columnToMap], point_radius=point_radius)
fig.add_layer(commute_heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))

In [295]:
# Change the heat map colors
commute_heatmap_layer.gradient = [(255,255,255,0),(0,0,255,0.2),(0,0,255,0.3),(0,0,255,0.4),(0,0,255,0.5),(0,0,255,0.6)]

In [297]:
embed_minimal_html('./Visuals/commuteScoreHeatMap.html', views=[fig])

## Property Value Analysis


In [298]:
# Troy's section
# This section will prepare the data and display on a heatmap

# read in masterdata file into dataframe

#data_df = pd.read_csv(masterDataCLEAN_csv)

data_df = data_df.rename(columns={'value sqft': 'valueSqft'})
min = data_df.valueSqft.min()
max = data_df.valueSqft.max()

data_df["valueScore"] =100-((data_df.valueSqft -min)*(100/(max-min)))

data_df.head()


,Unnamed: 0,Zillow ID,address,zipcode,alat,alon,valuation,sqft,valueSqft,neighborhood,tractCode,countyFips,stateFips,commuteTime,Region,Severity,commuteScore,valueScore,saftyScore,safetyScore
0,0,29498621,8113 West Gate Boulevard,78745,30.197589,-97.828106,265572,940,282.52,Cherry Creek,1728,453,48,27.7,H7,476,46.048110,82.485340,91.346329,91.346329
1,1,29440641,2109 Red Stone Lane,78727,30.418454,-97.696190,249696,1359,183.74,Tomanet Estates,1848,453,48,21.4,O12,970,67.697595,89.220646,82.346511,82.346511
2,2,29345444,9 Glen Rock Drive,78738,30.341187,-97.984049,472040,2896,163.00,NaN,1773,453,48,30.8,M2,0,35.395189,90.634802,100.018218,100.018218
3,3,29613796,8333 Alvin High Lane,78729,30.460441,-97.761118,307384,1983,155.01,Hunter's Chase,20411,491,48,27.0,Q10,62,48.453608,91.179599,98.888686,98.888686
4,4,29358809,5300 Valburn Circle,78731,30.374096,-97.775259,941669,4376,215.19,North Cat Mountain,1705,453,48,20.3,N9,300,71.477663,87.076231,94.552742,94.552742


In [299]:
# Creating a heat map for the property value/sqft

df = data_df
columnToMap = 'valueSqft'
point_radius = 10

figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}


fig = gmaps.figure(layout=figure_layout)
value_heatmap_layer = gmaps.heatmap_layer(df[['alat', 'alon']], weights=df[columnToMap], point_radius=point_radius)
fig.add_layer(value_heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))

In [300]:
# Change the heat map colors
value_heatmap_layer.gradient = [(255,255,255,0),(255,0,0,0.2),(255,0,0,0.3),(255,0,0,0.4),(255,0,0,0.5),(255,0,0,0.6)]

In [301]:
embed_minimal_html('./Visuals/valuesqftHeatMap.html', views=[fig])

## Value vs Commute Analysis

In [303]:

fig = gmaps.figure(layout=figure_layout)
fig.add_layer(commute_heatmap_layer)
fig.add_layer(value_heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))

In [254]:
embed_minimal_html('./Visuals/valuesvscommuteHeatMap.html', views=[fig])

## Crime Analysis

In [308]:
# Troy's section
# This section will prepare the data and display on a heatmap

# read in masterdata file into dataframe

#data_df = pd.read_csv(masterDataCLEAN_csv)

min = data_df.Severity.min()+1
max = data_df.Severity.max()

data_df["safetyScore"] =100-((data_df.Severity -min)*(100/(max-min)))

data_df


,Unnamed: 0,Zillow ID,address,zipcode,alat,alon,valuation,sqft,valueSqft,neighborhood,tractCode,countyFips,stateFips,commuteTime,Region,Severity,commuteScore,valueScore,saftyScore,safetyScore
0,0,29498621,8113 West Gate Boulevard,78745,30.197589,-97.828106,265572,940,282.52,Cherry Creek,1728,453,48,27.7,H7,476,46.048110,82.485340,91.346329,91.346329
1,1,29440641,2109 Red Stone Lane,78727,30.418454,-97.696190,249696,1359,183.74,Tomanet Estates,1848,453,48,21.4,O12,970,67.697595,89.220646,82.346511,82.346511
2,2,29345444,9 Glen Rock Drive,78738,30.341187,-97.984049,472040,2896,163.00,NaN,1773,453,48,30.8,M2,0,35.395189,90.634802,100.018218,100.018218
3,3,29613796,8333 Alvin High Lane,78729,30.460441,-97.761118,307384,1983,155.01,Hunter's Chase,20411,491,48,27.0,Q10,62,48.453608,91.179599,98.888686,98.888686
4,4,29358809,5300 Valburn Circle,78731,30.374096,-97.775259,941669,4376,215.19,North Cat Mountain,1705,453,48,20.3,N9,300,71.477663,87.076231,94.552742,94.552742
5,5,29471589,11125 Maha Loop Road,78617,30.082419,-97.664366,212919,1960,108.63,NaN,2436,453,48,26.7,D13,0,49.484536,94.342016,100.018218,100.018218
6,6,29352791,102 Javelin Drive,78734,30.360267,-97.998646,370772,2088,177.57,NaN,1771,453,48,34.7,M2,0,21.993127,89.641347,100.018218,100.018218
7,7,64884422,7220 Cielo Azul Pass,78726,30.416620,-97.874616,2592410,6060,427.79,NaN,1765,453,48,31.3,O6,0,33.676976,72.580117,100.018218,100.018218
8,8,29417097,2000 Brentwood Street,78757,30.340507,-97.734626,445342,1051,423.73,Abercrombie,1505,453,48,26.5,M11,1728,50.171821,72.856948,68.537074,68.537074
9,9,144341633,1253 Dunlap Road South,78725,30.222870,-97.566622,361665,2550,141.83,Chaparral Crossing,2207,453,48,36.5,I17,0,15.807560,92.078276,100.018218,100.018218


In [309]:
# Creating a heat map for the safty score

df = data_df
columnToMap = 'safetyScore'
point_radius = 10

figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}


fig = gmaps.figure(layout=figure_layout)
safety_heatmap_layer = gmaps.heatmap_layer(df[['alat', 'alon']], weights=df[columnToMap], point_radius=point_radius)
fig.add_layer(safety_heatmap_layer)

fig


Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))

In [310]:
# Change the heat map colors
safety_heatmap_layer.gradient = [(255,255,255,0),(0,0,255,0.2),(0,0,255,0.3),(0,0,255,0.4),(0,0,255,0.5),(0,0,255,0.6)]

In [311]:
embed_minimal_html('./Visuals/saftyHeatMap.html', views=[fig])

## Value vs Safety Analysis

In [284]:
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(safety_heatmap_layer)
fig.add_layer(value_heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))

In [312]:
embed_minimal_html('./Visuals/safetyvscommuteHeatMap.html', views=[fig])

## Regression Analysis

In [290]:
import statsmodels.formula.api as sm
#data_df = pd.read_csv(masterDataCLEAN_csv)

result = sm.ols(formula="valueScore ~ commuteScore + safetyScore", data=data_df).fit()
print(result.params)

Intercept       87.099765
commuteScore    -0.123948
safetyScore      0.061649
dtype: float64
